In [93]:
import re
from bs4 import BeautifulSoup
import requests
import json

# insurance

In [4]:
response = requests.get("http://localhost:8000/bank/insurance")
insurance_list = response.json()["items"]

In [6]:
sravni_list = requests.get(
    "https://www.sravni.ru/proxy-organizations/organizations?active=true&limit=1000&organizationType=insuranceCompany&skip=0"
).json()["items"]

In [37]:
sravni_insurance = []
for sravni in sravni_list:
    if len(sravni["license"]) == 0:
        continue
    license = re.findall("(?:(?<=№)|(?<=№\s))\d+(?:(?=\sот)|(?=-\d+|\s))", sravni["license"])[0]
    sravni_insurance.append({"id": sravni["id"], "alias": sravni["alias"], "name": sravni["name"], "licence": licence})

In [49]:
for sravni in sravni_insurance:
    fl = True
    for insurance in insurance_list:
        if int(sravni["licence"]) == int(insurance["licence"]):
            fl = False
            break
    if fl:
        print(sravni)

# broker
No reviews

In [50]:
response = requests.get("http://localhost:8000/bank/broker")
broker_list = response.json()["items"]

In [84]:
response = requests.post(
    "https://public.sravni.ru/v1/vitrins/products",
    json={
        "productName": "brokerskoe-obsluzhivanie",
        "limit": 200,
        "offset": 0,
        "advertisingOnly": False,
        "location": "6.83.",
        "offerTypes": [],
        "isMix": False,
        "group": ["organization"],
        "groupLimit": 5,
        "sortProperty": "advertising.position",
        "sortDirection": "asc",
    }
)
sravni_brokers = response.json()["items"]

# MFO

In [87]:
response = requests.get("https://www.sravni.ru/zaimy/mfo/")

In [91]:
page = BeautifulSoup(response.text, "html.parser")

In [92]:
js = page.find(id="__NEXT_DATA__")

In [97]:
js = json.loads(js.text)

In [116]:
sravmi_mfo = []
for mfo in js["props"]["initialReduxState"]["organizations"]["list"]:
    sravmi_mfo.append({
        "id": mfo["id"],
        "license": mfo["license"],
        "alias": mfo["alias"],
        "name": mfo["name"]["short"],
        "full_name": mfo["name"]["full"],
        "official_name": mfo["name"]["official"],
        "orgn": mfo["requisites"]["ogrn"]
    })

In [107]:
response = requests.get("http://localhost:8000/bank/mfo")
mfo_list = response.json()["items"]

In [123]:
c = 0
for sravni in sravmi_mfo:
    fl = True
    for mfo in mfo_list:
        description = json.loads(mfo["description"])
        if (int(sravni["license"]) == int(mfo["licence"])) or (int(sravni['orgn'])==int(description['ogrn'])):
            c += 1
            fl = False
            break
    if fl:
        print(sravni)
c

{'id': '6236234da1453800245283a7', 'license': '0', 'alias': 'dzhomin', 'name': 'Джомин', 'full_name': 'ООО «Джомин»', 'official_name': 'Общество с ограниченной ответственностью «Джомин»', 'orgn': '1186451027014'}


273

In [113]:
len(sravmi_mfo)

274

In [121]:
mfo

{'id': 11382,
 'bank_name': 'Общество с ограниченной ответственностью Микрокредитная компания «Диджитал Мани»',
 'licence': '2004150009570',
 'description': '{"ogrn": 1195476084968, "short_name": "\\u041e\\u041e\\u041e \\u041c\\u041a\\u041a \\u00ab\\u0414\\u0438\\u0434\\u0436\\u0438\\u0442\\u0430\\u043b \\u041c\\u0430\\u043d\\u0438\\u00bb"}'}

In [122]:
sravni

{'id': '6328952a3ae9540023167114',
 'license': '2004150009570',
 'alias': 'didzhital-mani',
 'name': 'Диджитал Мани',
 'full_name': 'ООО МКК «Диджитал Мани»',
 'official_name': 'Общество с ограниченной ответственностью Микрокредитная компания «Диджитал Мани»',
 'orgn': '1195476084968'}